In [1]:
import requests
from nba_api.stats.endpoints import commonplayerinfo
from nba_api.stats.static import players
from nba_api.stats.endpoints import playercareerstats


#beginDate is the day before the matchup and endDate is the day after final day of matchup
def getScheduleMap(beginDate, endDate):
    
    scheduleResponse = requests.get("https://data.nba.com/data/10s/v2015/json/mobile_teams/nba/2020/league/00_full_schedule.json")
    
    #update schedule variable if the month is changing
    schedule = scheduleResponse.json()['lscd'][0]['mscd']['g']
    
    matchupGameCount ={'ATL':0, 'BKN':0, 'BOS':0,'CHA':0,'CHI':0,'CLE':0, 'DAL':0,
                   'DEN':0, 'DET':0, 'GSW':0,'HOU':0,'IND':0,'LAC':0, 'LAL':0,
                   'MEM':0, 'MIA':0, 'MIL':0,'MIN':0,'NOP':0,'NYK':0, 'OKC':0,
                   'ORL':0, 'PHI':0, 'PHX':0,'POR':0,'SAC':0,'SAS':0, 'TOR':0,
                   'UTA':0, 'WAS':0}

    for x in schedule:
        if x['gdte']>beginDate and x['gdte']<endDate:
            matchupGameCount[x['v']['ta']]+=1
            matchupGameCount[x['h']['ta']]+=1
    return matchupGameCount


def matchupPlayerProjections(fullName,matchupGameCount,statDict):
    
    #rookies are currently broken, hoping after first games are played things will be fixed
    rookieDict = {'LaMelo Ball': 1630163, 'James Wiseman': 1630164, 'Tyrese Haliburton' : 1630169}
    if not fullName in rookieDict :
        playerId = players.find_players_by_full_name(fullName)[0]['id']
        #grabs teamID for matchup multiplier
        player_info = commonplayerinfo.CommonPlayerInfo(player_id=str(playerId))
        teamId = player_info.common_player_info.get_dict()['data'][0][20]
        
    else:
        player_info = commonplayerinfo.CommonPlayerInfo(player_id=str(rookieDict[fullName]))
        teamId = player_info.common_player_info.get_dict()['data'][0][20]
    

    multiplier = matchupGameCount[teamId]

    projStats = {}
    
    projStats['reb']=(statDict['6'])*multiplier
    projStats['ast']=(statDict['26'])*multiplier
    projStats['stl']=(statDict['31'])*multiplier
    projStats['blk']=(statDict['27'])*multiplier
    projStats['pts']=(statDict['0'])*multiplier
    projStats['fta']=(statDict['16'])*multiplier
    projStats['ftm']=(statDict['15'])*multiplier
    projStats['fga']=(statDict['14'])*multiplier
    projStats['fgm']=(statDict['13'])*multiplier
    if '33' in statDict:
        projStats['3pm']=(statDict['33'])*multiplier
    else:
        projStats['3pm']=0
        
    return projStats

def generateTeamStatLine(roster,matchupGameCount):
    statList=[]
    injList={}
    rookieDict = {'LaMelo Ball': 1630163, 'James Wiseman': 1630164, 'Tyrese Haliburton' : 1630169}
    for x in  roster:
        fullName = x['playerPoolEntry']['player']['fullName']
        print(fullName)
        if fullName in rookieDict:
            statDict = x['playerPoolEntry']['player']['stats'][3]['averageStats']
        else:
            statDict = x['playerPoolEntry']['player']['stats'][4]['averageStats']
        #checks injured status of player
        if x['playerPoolEntry']['player']['injuryStatus']=='OUT':
            injList[fullName]='OUT'
        #rookies currently break analysis
        if not (fullName in injList):
            projStats = (matchupPlayerProjections(fullName, matchupGameCount,statDict))
            statList.append(projStats)
    return statList
            
    
def statLineTotaler(statList,totalStats):
    for x in statList:
        totalStats['reb']+=x['reb']
        totalStats['ast']+=x['ast']
        totalStats['stl']+=x['stl']
        totalStats['blk']+=x['blk']
        totalStats['pts']+=x['pts']
        totalStats['fta']+=x['fta']
        totalStats['ftm']+=x['ftm']
        totalStats['fga']+=x['fga']
        totalStats['fgm']+=x['fgm']
        totalStats['3pm']+=x['3pm']
        
    totalStats['ftp']=totalStats['ftm']/totalStats['fta']    
    totalStats['fgp']=totalStats['fgm']/totalStats['fga']
    
    return totalStats
    
    
def calculateMatchupResults(totalHomeStats, totalVisitStats):
    statList = ['reb','ast','stl','blk','pts','ftp','fgp','3pm']
    resultMap ={}
    homeScore=0
    visitScore=0
    
    for x in statList:
        if totalHomeStats[x]>totalVisitStats[x]:
            resultMap[x]='Home'
            homeScore+=1
        else:
            resultMap[x]='Visit'
            visitScore+=1
    print('Home Score is:'+str(homeScore))
    print('Visit Score is:'+str(visitScore))
    
    return resultMap
    
    
def calculateTeamProjection(teamId,matchupGameCount):
    #get roster of current team
    response = requests.get("https://fantasy.espn.com/apis/v3/games/fba/seasons/2021/segments/0/leagues/55912505?rosterForTeamId="+str(teamId)+"&view=mRoster")
    roster = response.json()['teams'][teamId-1]['roster']['entries']
    
    #prep stats for storage
    totalStats= {'reb':0, 'ast': 0, 'stl': 0, 'blk':0,'pts':0, 'fta':0, 'ftm':0, 'fga':0, 'fgm':0, '3pm':0}
    
    #create list of each players projected stats
    statList = generateTeamStatline(roster,matchupGameCount)
    
    #totals each teams stats
    totalStats = statLineTotaler(statList,totalStats)
    
    return totalStats
    
    
    

#Brenneman's team id = 1
#      Cob's team id = 2
#  Mahoney's team id = 3
#  Sharkey's team id = 4
#     Luke's team id = 5
#   Victor's team id = 6
#   Trevor's team id = 7
#   Pehuen's team id = 8
def matchupProjection(beginDate, endDate, homeTeamId, visitTeamId):

    #create schedule for matchup
    matchupGameCount = getScheduleMap(beginDate,endDate)
    
    totalHomeStats = calculateTeamProjection(homeTeamId,matchupGameCount)
    totalVisitStats= calculateTeamProjection(visitTeamId,matchupGameCount)

    results = calculateMatchupResults(totalHomeStats,totalVisitStats)
    print(totalHomeStats)
    print(totalVisitStats)
    print (results)
